## This file is for taking nhd centerlines and creating polygons around unique reaches (COMIDs)

Previously, this approach was done with "collapsed" nhd centerlines, where some centerlines were merched together to reduce the number of IDs

In [6]:
import ee
ee.Initialize()
import time
import ee
import os

In [7]:
# Here are the 'un collapsed' centerlines from nhd 
all_nhd_centerlines = ee.FeatureCollection("projects/ee-samsillen0/assets/nhd_grwl_full")


In [8]:
HUC04 = ee.FeatureCollection("USGS/WBD/2017/HUC04")

#HUC08 = ee.FeatureCollection("USGS/WBD/2017/HUC08")

### define functions for finding pixels  closest to each reach in centerline file
# change the "ID" to whatever the unique ID is of each reach

def fdtFun(f):
    dt = ee.Image.constant(0).byte().paint(ee.FeatureCollection(f), 1).fastDistanceTransform()
    return dt.copyProperties(f, ['ID'])

def mfun(c, p):
    return ee.Image(c).min(p)

def minFun(f):
  return ee.Image(f.iterate(mfun, ee.Image(f.first())))

def idFun(c, p):
    c = ee.Image(c)
    p = ee.Image(p)
    return p.where(c.eq(minMap), ee.Number(c.get('ID')))


In [9]:
# functions for bacth processing
def maximum_no_of_tasks(MaxNActive, waitingPeriod):
  ##maintain a maximum number of active tasks
  time.sleep(10)
  ## initialize submitting jobs
  ts = list(ee.batch.Task.list())

  NActive = 0
  for task in ts:
       if ('RUNNING' in str(task) or 'READY' in str(task)):
           NActive += 1
  ## wait if the number of current active tasks reach the maximum number
  ## defined in MaxNActive
  while (NActive >= MaxNActive):
      time.sleep(waitingPeriod) # if reach or over maximum no. of active tasks, wait for 2min and check again
      ts = list(ee.batch.Task.list())
      NActive = 0
      for task in ts:
        if ('RUNNING' in str(task) or 'READY' in str(task)):
          NActive += 1
  return()


In [11]:
## loop through all HUC04 and output reach 
hucID = HUC04.aggregate_array('huc4').getInfo()


for x in range(0,len(hucID)):
#for x in range(0,2):
    
    #huc = HUC04.filter(ee.Filter.eq('huc4' , hucID[x]))
    huc = HUC04.filter(ee.Filter.stringStartsWith('huc4' , hucID[x]))
    
    nhdClip = all_nhd_centerlines.filterBounds(huc.geometry())

# set maximum buffer from center line to assign pixels a reach ID (2000 m)
    # when I do global pull I am going to jack this up to 5000-10000
    nhdClip_buffer = nhdClip.geometry().buffer(2000)

##
    fdt = nhdClip.map(fdtFun)
    
    minMap = minFun(fdt)
    
    idImage = fdt.iterate(idFun, ee.Image(fdt.first()))
    
# make image and clip to max distance buffer from centerline for vector export
    COMID = ee.Image(idImage).rename('ID').clip(nhdClip_buffer)
##
    vectors = COMID.addBands(COMID).reduceToVectors(
        crs = COMID.projection(), 
        scale =  60, 
        geometry = huc, 
        geometryType = 'polygon',
        eightConnected= False,
        tileScale = 16,
        bestEffort = True,
        labelProperty = 'ID',
        maxPixels = 3000000000000000, 
        reducer = ee.Reducer.first()
        )

    
    dataOut = ee.batch.Export.table.toDrive(collection = vectors, \
                                              description = str(hucID[x]),\
                                              folder = 'reachPolygons',\
                                              fileFormat = 'shp')
    
    print(hucID[x])
  #Check how many existing tasks are running and take a break if it's >15  
    maximum_no_of_tasks(15, 60)
  #Send next task.
    dataOut.start()

#Make sure all Earth engine tasks are completed prior to moving on.  
maximum_no_of_tasks(1,300)
print('done')

0101
0102
0103
0318
0104
0402
0401
1903
1905
2203
1507
1506
1805
1806
1803
2201
1206
1902
0804
0805
1101
1111
1505
1504
1502
1407
1807
1810
1808
1801
0408
0407
1103
1302
1301
0709
1029
1106
1105
0807
0806
0403
1308
1309
1016
1205
1208
1305
1306
1307
0802
1508
1026
1108
1404
1304
1303
1015
1109
1112
1107
0803
1503
1408
1104
0708
1602
0406
1110
1011
1012
0801
1709
1703
1702
1711
1708
1707
1712
1710
1705
1701
1706
1909
1202
1113
1203
1207
1201
1114
0903
0901
0902
0202
1401
1405
1906
1907
1908
0808
1204
1210
0107
0106
1403
1402
0105
2202
1211
0809
1209
1017
1003
1014
1013
1102
2003
2004
2002
2007
2008
2005
2006
2001
1028
1030
1022
1024
1020
1027
1021
1010
1006
1002
1704
1007
1004
1009
1008
1023
0711
1019
1018
1025
1005
0904
1601
1501
1809
1606
1603
1406
0702
0710
0714
1901
0701
0703
0707
0704
0706
0705
1802
1804
1604
1605
0713
0108
0415
0315
0316
0206
0414
0508
0410
0409
0412
0501
0411
0504
0503
0502
0506
0510
0511
0513
0601
0602
0603
0604
0509
2102
0306
0305
0110
0205
2101
0207
0505
0507
